# Lab 7: Word Sequences
## Introduction to Human Language Technologies
### Victor Badenas Crespo

***

## Statement:

- Read all pairs of sentences of the trial set within the evaluation framework of the project.
- Compute their similarities by considering the following approach:
    - words plus NEs and Jaccard coefficient ex: word_and_NEs=\['John Smith', 'is', 'working'\]
- Show the results.
- Do you think it could be relevant to use NEs to compute the similarity between two sentences? Justify the answer.



***

## Solution

In [1]:
# core imports
from pprint import pprint
from pathlib import Path
from collections import Counter

# scipy imports
from scipy.stats import pearsonr

# nltk imports
import nltk
from nltk.metrics import jaccard_distance
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk

# nltk downloads
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('conll2000')

# constants definition
DATA_FOLDER = Path('./trial')

[nltk_data] Downloading package punkt to /Users/victor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /Users/victor/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/victor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/victor/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/victor/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


First functions for reading and structuring the data are declared, then the input data is read which has multiple lines containing \[id, sentence1, sentence2\]. The Gold standard info is also read. Then the inputText is formatted into a dict object with the following format for readability:

```json
{
    "id": <id_string>,
    "sent1": <sentence_string>,
    "sent2": <sentence_string>
}
```

In [2]:
def readFile(filePath):
    """
    reads and returns a list of lists containing the text split by line 
    jumps and by tab characters
    """
    with open(filePath, 'r') as fileHandler:
        data = fileHandler.readlines()
    
    # split every line by tabs
    data = list(map(lambda x: x.strip().split('\t'), data))
    return data

def toDict(line):
    """
    creates a dict with fields id sent1 sent2 from the values in line
    """
    keys = ("id", "sent1", "sent2")
    return dict(zip(keys, line))

# read file data
inputText = readFile(DATA_FOLDER / 'STS.input.txt')
gsText = readFile(DATA_FOLDER / 'STS.gs.txt')

# convert to previously defined dict structure
inputText = list(map(toDict, inputText))
pprint(inputText)

[{'id': 'id1',
  'sent1': 'The bird is bathing in the sink.',
  'sent2': 'Birdie is washing itself in the water basin.'},
 {'id': 'id2',
  'sent1': 'In May 2010, the troops attempted to invade Kabul.',
  'sent2': 'The US army invaded Kabul on May 7th last year, 2010.'},
 {'id': 'id3',
  'sent1': 'John said he is considered a witness but not a suspect.',
  'sent2': '"He is not a suspect anymore." John said.'},
 {'id': 'id4',
  'sent1': 'They flew out of the nest in groups.',
  'sent2': 'They flew into the nest together.'},
 {'id': 'id5',
  'sent1': 'The woman is playing the violin.',
  'sent2': 'The young lady enjoys listening to the guitar.'},
 {'id': 'id6',
  'sent1': 'John went horse back riding at dawn with a whole group of friends.',
  'sent2': 'Sunrise at dawn is a magnificent view to take in if you wake up '
           'early enough for it.'}]


In [3]:
for sentence in inputText:
    id, sent1, sent2 = sentence["id"], sentence["sent1"], sentence["sent2"]
    sent1 = nltk.word_tokenize(sent1)
    sent2 = nltk.word_tokenize(sent2)
    t_POS_sent1 = nltk.pos_tag(sent1)
    t_POS_sent2 = nltk.pos_tag(sent2)
    sentence["ne_chunk1"] = nltk.ne_chunk(t_POS_sent1)
    sentence["ne_chunk2"] = nltk.ne_chunk(t_POS_sent2)

In [4]:
pprint(inputText[0])

{'id': 'id1',
 'ne_chunk1': Tree('S', [('The', 'DT'), ('bird', 'NN'), ('is', 'VBZ'), ('bathing', 'VBG'), ('in', 'IN'), ('the', 'DT'), ('sink', 'NN'), ('.', '.')]),
 'ne_chunk2': Tree('S', [Tree('GPE', [('Birdie', 'NNP')]), ('is', 'VBZ'), ('washing', 'VBG'), ('itself', 'PRP'), ('in', 'IN'), ('the', 'DT'), ('water', 'NN'), ('basin', 'NN'), ('.', '.')]),
 'sent1': 'The bird is bathing in the sink.',
 'sent2': 'Birdie is washing itself in the water basin.'}


In [5]:
def computeSimilarity(sentenceDict):
    context1 = set(sentenceDict["ne_chunk1"])
    context2 = set(sentenceDict["ne_chunk2"])
    return jaccard_distance(context1, context2)

In [6]:
testDistances = list(map(computeSimilarity, inputText))
refDistances = [float(value) for _, value in gsText]

pcorr = pearsonr(refDistances, testDistances)[0]

# formatting for purely demonstrative purposes
print(f"pearsonr({list(map(lambda x:float('%.2f' % x), refDistances))}, {list(map(lambda x:float('%.2f' % x), testDistances))}) = {pcorr}")

TypeError: unhashable type: 'Tree'

***

## Conlusion

***

### End of P4